# Data Science template - Titanic Case

Deze notebook begeleidt je stap voor stap door de **Titanic**-case op Kaggle.
De focus ligt op de standaard Data Science workflow:

1. Probleemdefinitie en doel
2. Data-input & Data cleaning
3. Verkenning (EDA)
4. Feature engineering
5. Modelleren (baseline → verbeterde modellen)
6. **Voorspelling** en export van `submission.csv`

> 🎯 Doel: voorspel de kans dat een passagier de Titanic-vaart heeft overleefd (`Survived`).

## 0. Doel & Omgevings-setup

- Download `train.csv` en `test.csv` handmatig van de Titanic-competitiepagina en plaats ze in een bijvoorbeeld in dezelfde map als dit bestand.


In [1]:
# !python --version


## 1. Imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## 2. Probleemdefinitie

- **Input**: passagiersgegevens zoals `Pclass`, `Sex`, `Age`, `SibSp`, `Parch`, `Fare`, `Embarked`, etc.
- **Output**: binaire target `Survived` (0 = niet overleefd, 1 = overleefd).
- **Metric (Kaggle leaderboard)**: **Accuracy** op de testset (zonder labels).


## 3. Data inladen

In [9]:
# Je gaat werken met de train data. In de test data staan de Survived kolom niet ingevuld. Deze moet je dus zelf voorspellen.

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

print(train.isna().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


## 4. Snelle datacheck & Preprocessing

Waar missen we (rijen met) data? Je kan missende data invullen door bijvoorbeeld voor bepaalde soort data op een bepaalde manier in te vullen:

- Numeriek: imputatie (median) + scaling
- Categorisch: imputatie (most_frequent) + one-hot encoding

In [ ]:
# Bereken gemiddelde en standaarddeviatie van bestaande leeftijden
mean_age = train["Age"].mean()
std_age = train["Age"].std()

# Vul missende leeftijden in met normale verdeling en clamp tussen 0 en 81
train["Age"] = train["Age"].apply(
    lambda x: int(np.clip(np.random.normal(mean_age, std_age), 0, 81)) if pd.isna(x) else x
)

# Bereken gemiddelde en standaarddeviatie van bestaande leeftijden
mean_age = test["Age"].mean()
std_age = test["Age"].std()

# Vul missende leeftijden in met normale verdeling en clamp tussen 0 en 81
test["Age"] = test["Age"].apply(
    lambda x: int(np.clip(np.random.normal(mean_age, std_age), 0, 81)) if pd.isna(x) else x
)

# maak een dictionaries van Sex en Embarked
dict_Embarked = {'S':0, 'C':1, 'Q':2} 
dict_Sex = {"male": 0, "female": 1}
dict_Pclass = {1:0, 2:1, 3:2}

# drop missing values of column 'Embarked'
train = train.dropna(subset=['Embarked'])

# maak nieuwe kolommen voor sex en Embarked
train['Embarked_int'] = train['Embarked'].map(dict_Embarked)
train["Sex_int"] = train["Sex"].map(dict_Sex)
train['Pclass_0'] = train['Pclass'].map(dict_Pclass)

train = train.set_index('PassengerId')

test = test.dropna(subset=['Embarked'])

test['Embarked_int'] = test['Embarked'].map(dict_Embarked)
test["Sex_int"] = test["Sex"].map(dict_Sex)
test['Pclass_0'] = test['Pclass'].map(dict_Pclass)

test = test.set_index('PassengerId')

### PRINT ALLE NANS
print(train.isna().sum())

survival_rate = train['Survived'].value_counts(normalize=True)[1]

def make_survivle_table(col, bins):
    bins = pd.cut(train[col], bins=bins, right=False)
    counts = bins.value_counts().sort_index()
    probs  = train.groupby(bins, observed=True)['Survived'].mean().reindex(counts.index)
    probs = probs.fillna(survival_rate)

    temp_df = pd.DataFrame({
      'bin': counts.index,
      'prob': probs.values,
      'count': counts.values
    }, index=[col]*len(counts))

    return temp_df

def qmake_survivle_table(col, q):
    bins = pd.qcut(train[col], q=q, duplicates='drop')

    counts = bins.value_counts().sort_index()
    probs  = train.groupby(bins, observed=True)['Survived'].mean().reindex(counts.index)
    probs = probs.fillna(survival_rate)

    temp_df = pd.DataFrame({
        'bin': counts.index,
        'prob': probs.values,
        'count': counts.values
    }, index=[col]*len(counts))

    return temp_df


bSurvival_rates_lookup = pd.DataFrame()

Age_bins_n = make_survivle_table('Age', range(0, 86, 5))
Fare_bins_n = make_survivle_table('Fare', range(0, int(train['Fare'].max()) + 5, 5))
Sex_bins_n = make_survivle_table('Sex_int', range(0, 3, 1))
SibSp_bins_n = make_survivle_table('SibSp', range(0, 7, 1))
Parch_bins_n = make_survivle_table('Parch', range(0, 8, 1))
Embarked_bins_n = make_survivle_table('Embarked_int', range(0, 4, 1))
Pclass_bins_n = make_survivle_table('Pclass_0', range(0, 4, 1))

Survival_rates_lookup = pd.concat([bSurvival_rates_lookup,
                                    Age_bins_n, 
                                    Fare_bins_n,
                                    Sex_bins_n,
                                    SibSp_bins_n,
                                    Parch_bins_n,
                                    Embarked_bins_n,
                                    Pclass_bins_n])

#print(Survival_rates_lookup[Survival_rates_lookup.index == 'Age'])


qAge_bins_n = qmake_survivle_table('Age', 10)
qFare_bins_n = qmake_survivle_table('Fare', 25)
qSex_bins_n = make_survivle_table('Sex_int', range(0, 3, 1))
qSibSp_bins_n = make_survivle_table('SibSp', range(0, 8, 1))
qParch_bins_n = make_survivle_table('Parch', range(0, 8, 1))
qEmbarked_bins_n = make_survivle_table('Embarked_int', range(0, 4, 1))
qPclass_bins_n = make_survivle_table('Pclass_0', range(0, 4, 1))

qSurvival_rates_lookup = pd.concat([bSurvival_rates_lookup,
                                    qAge_bins_n, 
                                    qFare_bins_n,
                                    qSex_bins_n,
                                    qSibSp_bins_n,
                                    qParch_bins_n,
                                    qEmbarked_bins_n,
                                    qPclass_bins_n])


Survived          0
Pclass            0
Name              0
Sex               0
Age               0
SibSp             0
Parch             0
Ticket            0
Fare              0
Cabin           687
Embarked          0
Embarked_int      0
Sex_int           0
Pclass_0          0
dtype: int64
                bin      prob  count
Age  (-0.001, 11.8]  0.516854     89
Age    (11.8, 18.0]  0.400000     90
Age    (18.0, 22.0]  0.302752    109
Age    (22.0, 25.0]  0.361446     83
Age    (25.0, 29.0]  0.350515     97
Age    (29.0, 32.0]  0.415584     77
Age    (32.0, 36.0]  0.451613     93
Age    (36.0, 41.0]  0.315789     76
Age    (41.0, 50.0]  0.389474     95
Age    (50.0, 80.0]  0.325000     80


C:\Users\guush\AppData\Local\Temp\ipykernel_18220\848345916.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['Embarked_int'] = train['Embarked'].map(dict_Embarked)
C:\Users\guush\AppData\Local\Temp\ipykernel_18220\848345916.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["Sex_int"] = train["Sex"].map(dict_Sex)
C:\Users\guush\AppData\Local\Temp\ipykernel_18220\848345916.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

## 5. Verkennende Data Analyse (EDA)

> Tip: Voeg hier extra grafieken toe (bijv. histogrammen, boxplots, countplots per feature).


In [ ]:
# Convert Series to DataFrame
#df = FS_n[1].reset_index()
#df.columns = ["Age", "SurvivalChance"]

# Choose bin size (e.g., 10 years)
#bin_size = 1
#bins = range(0, int(df["Age"].max()) + bin_size, bin_size)

# Cut ages into bins and compute mean survival chance
#df["AgeBin"] = pd.cut(df["Age"], bins=bins, right=False)
#survival_by_bin = df.groupby("AgeBin")["SurvivalChance"].mean()

# Plot histogram (bar chart of averages per bin)
#plt.figure(figsize=(10,5))
#survival_by_bin.plot(kind="bar", width=0.8)
#plt.xlabel("Age Group")
#plt.ylabel("Average Survival Chance")
#plt.title(f"Average Survival Chance per {bin_size}-Year Age Bin")
#plt.show()


NameError: name 'FS_n' is not defined

In [ ]:
# Oefening: voeg je eigen EDA toe
# Gebruik Seaborn in plaats van Matplotlib.
# TODO: tel bijvoorbeeld overlevingspercentage per Pclass, Embarked, Fare-quantiles (pd.qcut), etc.

## 6. Feature Engineering

Voeg hier zelf extra features toe (bijv. `Title` uit `Name`, `FamilySize = SibSp + Parch + 1`, indicatoren, etc.).

Zorg wel dat het relevant is voor je eigen onderzoek. Waar ligt je eigen focus.

In [ ]:
# Je kan nu zelf features aanmaken invullen. 
train['FamilySize'] = train['SibSp'] + train['Parch'] + 1

print(train['FamilySize'])
FS_n = pd.crosstab(train['FamilySize'], train['Survived'], normalize='index')


''' 
# maak een dictionaries van Sex en Embarked
dict_Embarked = {'S':0, 'C':1, 'Q':2} 
dict_Sex = {"male": 0, "female": 1}
dict_Pclass = {1:0, 2:1, 3:2}

# drop missing values of column 'Embarked'
train = train.dropna(subset=['Embarked'])

# maak nieuwe kolommen voor sex en Embarked
train['Embarked_int'] = train['Embarked'].map(dict_Embarked)
train["Sex_int"] = train["Sex"].map(dict_Sex)
train['Pclass_0'] = train['Pclass'].map(dict_Pclass)

train = train.dropna(subset=['Embarked'])
'''

0      2
1      2
2      1
3      2
4      1
      ..
886    1
887    1
888    4
889    1
890    1
Name: FamilySize, Length: 889, dtype: int64


## 7. Baseline model — np.where invullen

In [54]:
'''
DataFrame SurvivalProbability

index = personID   pPclass  pSex  pEmbarked  pTotal

hier gaan we eerst p per persoon bepalen

returnen:
persoonID
pPclass
pSex
pEmbarked
pTotal

pTotal = log(pPclass) + log(pSex) + log(pEmbarked)  // scaled or calculated odds = np.exp(logP_survive - logP_die)  prob_survive = odds / (1 + odds)   // 
pTotal = pPclass * pSex * pEmbarked


?pTotalScaled?

'''

def lookup_probability(index, value): # index = Sex, Pclass, Embarked
    feature_rows = qSurvival_rates_lookup.loc[index]
    row = feature_rows[feature_rows['bin'].apply(lambda x: value in x)]

    if not row.empty:
        if row['prob'].iloc[0] <= 1e-3:
            return 1e-3
        else:
            return row['prob'].iloc[0]
    else:
        return survival_rate


In [53]:
Age_weight = .7
Fare_weight = 0.2
Sex_weight = 1.5
SibSp_weight = 1
Parch_weight = 1.05
Emabrked_weight = 1
Class_weight = .0

SurvivalTest = pd.DataFrame(index=train.index)
SurvivalTest['pAge'] = train['Age'].apply(lambda v: lookup_probability('Age', v))
SurvivalTest['pFare'] = train['Fare'].apply(lambda v: lookup_probability('Fare', v))
SurvivalTest['pSex'] = train['Sex_int'].apply(lambda v: lookup_probability('Sex_int', v))
SurvivalTest['pSibSp'] = train['SibSp'].apply(lambda v: lookup_probability('SibSp', v)) 
SurvivalTest['pParch'] = train['Parch'].apply(lambda v: lookup_probability('Parch', v))
SurvivalTest['pEmbarked'] = train['Embarked_int'].apply(lambda v: lookup_probability('Embarked_int', v))
SurvivalTest['pPclass'] = train['Pclass_0'].apply(lambda v: lookup_probability('Pclass_0', v))

SurvivalTest['pSurvive'] = (
    Age_weight *  np.log(SurvivalTest['pAge']) +
    Fare_weight * np.log(SurvivalTest['pFare']) +
    Sex_weight * np.log(SurvivalTest['pSex']) +
    SibSp_weight * np.log(SurvivalTest['pSibSp']) +
    Parch_weight * np.log(SurvivalTest['pParch']) +
    Emabrked_weight * np.log(SurvivalTest['pEmbarked']) +
    Class_weight * np.log(SurvivalTest['pPclass'])
)

SurvivalTest['pDie'] = (
    Age_weight * np.log(1 - SurvivalTest['pAge']) +
    Fare_weight * np.log(1 - SurvivalTest['pFare']) +
    Sex_weight * np.log(1 - SurvivalTest['pSex']) +
    SibSp_weight * np.log(1 - SurvivalTest['pSibSp']) +
    Parch_weight * np.log(1 - SurvivalTest['pParch']) +
    Emabrked_weight * np.log(1 - SurvivalTest['pEmbarked']) +
    Class_weight * np.log(1 - SurvivalTest['pPclass'])
)

SurvivalTest['pTotal'] = np.exp(SurvivalTest['pSurvive'] - SurvivalTest['pDie']) / (1 + np.exp(SurvivalTest['pSurvive'] - SurvivalTest['pDie']))

SurvivalTest['rank'] = SurvivalTest['pTotal'].rank(method='first', ascending=False)
n_survive = int(survival_rate * len(SurvivalTest)) + x
threshold_rank = n_survive
SurvivalTest['Survived_pred'] = (SurvivalTest['rank'] <= threshold_rank).astype(int)

### USE ON TRAIN DATA
SurvivalTest['Survived_true'] = train['Survived']
SurvivalTest['Wrong'] = SurvivalTest['Survived_pred'] != SurvivalTest['Survived_true']
wrong_predictions = SurvivalTest[SurvivalTest['Survived_pred'] != SurvivalTest['Survived_true']]

pred_fraction = SurvivalTest['Survived_pred'].mean()

pred_survive_fraction = SurvivalTest['Survived_pred'].sum() / len(SurvivalTest)

# Boolean column: True if prediction is wrong
SurvivalTest['Wrong'] = SurvivalTest['Survived_pred'] != SurvivalTest['Survived_true']

# Count how many are wrong
num_wrong = SurvivalTest['Wrong'].sum()

wrong_predictions = SurvivalTest[SurvivalTest['Wrong']]

# Count how many of the wrong predictions were actually survived (1) or died (0)
wrong_counts_by_true = wrong_predictions['Survived_true'].value_counts()

## 8. Maak een eindmodel op volledige trainingsdata en voorspel op test

Hier kiezen we als voorbeeld het np.where model (of gebruik anders een ander model).


In [50]:
# === Toepassen op testset ===

#submission = pd.DataFrame({
#    'PassengerId': test['PassengerId'],
#    'Survived': test_pred.astype(int)
#})

submission = pd.DataFrame({
    'PassengerId' : SurvivalTest.index,
    'Survived' : (SurvivalTest['pTotal'] > 0.5).astype(int)
})


submission.to_csv("submission.csv", index=False)
print("submission.csv aangemaakt!")
submission.head()


submission.csv aangemaakt!


,PassengerId,Survived
PassengerId,,
1,1,0
2,2,1
3,3,0
4,4,1
5,5,0


## 9. Checklist & Volgende stappen

- [ ] Zorg dat er een duidelijke samenhang is tussen de grafieken -> Feature Engineering -> je uiteindelijke model! Dat is het belangrijkst.
- [ ] Laat geen grafieken zien waar je niks mee doet tijdens de presentatie.
- [ ] Zorg dat er dus een rode draad is in je verhaal. 

Voorbeeld: We hebben de focus gelegd op leeftijd -> EDA laat deze grafieken met leeftijd zien. We hebben met Feature Engineering nog leeftijdsgroepen aangemaakt -> We hebben in het model de leeftijden gebruikt. Dit is onze score.

- [ ] Eventueel EDA (een klein beetje) uitbreiden (meer grafieken, correlaties, outliers)
- [ ] Feature engineering verbeteren (wat is er relevant?)
- [ ] Meer modellen proberen (Logistic Regression? Sklearn)
- [ ] Submission inleveren op kaggle en kijken hoe goed je score is

Succes met de case!